In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# pip install lxml

## Import Libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Turn to DataFrame

### Passing

#### Page 1

In [4]:
# Request raw html
passing1 = requests.get("https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(passing1.content, "html.parser")

# Inspect HTML code
#print(soup)

In [6]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df1 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df1.columns.difference(['Player'])
df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df1

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Tua Tagovailoa,466,10.4,45,28,62.2,3,1,110.0,23,51.1,8,1,47,0,0
1,Kirk Cousins,344,7.8,44,33,75.0,2,1,102.8,15,34.1,5,1,42,2,16
2,Matthew Stafford,334,8.8,38,24,63.2,0,0,91.3,17,44.7,6,1,44,0,0
3,Mac Jones,316,5.8,54,35,64.8,3,1,91.3,18,33.3,2,0,32,2,10
4,Derek Carr,305,9.2,33,23,69.7,1,1,96.2,12,36.4,5,2,45,4,23
5,Jared Goff,253,7.2,35,22,62.9,1,0,94.1,11,31.4,4,0,33,1,3
6,Jordan Love,245,9.1,27,15,55.6,3,0,123.2,10,37.0,4,1,51,1,8
7,C.J. Stroud,242,5.5,44,28,63.6,0,0,78.0,13,29.6,1,0,26,5,46
8,Trevor Lawrence,241,7.5,32,24,75.0,2,1,103.8,10,31.2,3,0,29,2,4
9,Kenny Pickett,232,5.0,46,31,67.4,1,2,68.4,9,19.6,2,0,31,5,34


#### Page 2

In [7]:
passing2 = requests.get("https://www.nfl.com/stats/player-stats/category/passing/2023/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAYeAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhORE1pTENJek1qQXdOVEExTWkwME5USTRMVFUzTWpNdFpERmlNaTA1Tm1VNU1tVmlZekV5TkRFaUxDSXlNREl6SWwxOQ==")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(passing2.content, "html.parser")


In [8]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df2 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df2.columns.difference(['Player'])
df2[numeric_columns] = df2[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df2

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Joshua Dobbs,132,4.4,30,21,70.0,0,0,78.8,9,30.0,1,0,31,3,18
1,Desmond Ridder,115,6.4,18,15,83.3,1,0,111.8,6,33.3,1,0,34,4,24
2,Geno Smith,112,4.3,26,16,61.5,1,0,84.1,7,26.9,1,0,28,2,17
3,Daniel Jones,104,3.7,28,15,53.6,0,2,32.4,5,17.9,2,0,24,7,47
4,Joe Burrow,82,2.6,31,14,45.2,0,0,52.2,3,9.7,0,0,12,2,15
5,Tyrod Taylor,6,3.0,2,2,100.0,0,0,79.2,0,0.0,0,0,9,0,0
6,Jake Browning,0,0.0,1,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0
7,Gardner Minshew,0,0.0,2,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0
8,Cooper Rush,0,0.0,1,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0


#### Combine Pages

In [9]:
# Stack them vertically (along rows)
passing = pd.concat([df1, df2])

# Sort by 'Pass Yds' in descending order
passing = passing.sort_values(by='Pass Yds', ascending=False)

# Reindex the dataframe
passing = passing.reset_index(drop=True)
passing

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Tua Tagovailoa,466,10.4,45,28,62.2,3,1,110.0,23,51.1,8,1,47,0,0
1,Kirk Cousins,344,7.8,44,33,75.0,2,1,102.8,15,34.1,5,1,42,2,16
2,Matthew Stafford,334,8.8,38,24,63.2,0,0,91.3,17,44.7,6,1,44,0,0
3,Mac Jones,316,5.8,54,35,64.8,3,1,91.3,18,33.3,2,0,32,2,10
4,Derek Carr,305,9.2,33,23,69.7,1,1,96.2,12,36.4,5,2,45,4,23
5,Jared Goff,253,7.2,35,22,62.9,1,0,94.1,11,31.4,4,0,33,1,3
6,Jordan Love,245,9.1,27,15,55.6,3,0,123.2,10,37.0,4,1,51,1,8
7,C.J. Stroud,242,5.5,44,28,63.6,0,0,78.0,13,29.6,1,0,26,5,46
8,Trevor Lawrence,241,7.5,32,24,75.0,2,1,103.8,10,31.2,3,0,29,2,4
9,Kenny Pickett,232,5.0,46,31,67.4,1,2,68.4,9,19.6,2,0,31,5,34


#### Export DataFrame

In [10]:
import os

## Check if directory "data" exists, if not create
# Define the directory path
directory_path = 'data'

# Check if the directory exists
if not os.path.exists(directory_path):
    # Create the directory if it doesn't exist
    os.makedirs(directory_path)
    print(f'Directory "{directory_path}" has been created.')
else:
    print(f'Directory "{directory_path}" already exists.')

Directory "data" has been created.


In [11]:
current_week = 'week1'

## Check if directory "week1" exists, if not create
# Combine the base directory path with the current week
directory_path = os.path.join('data', current_week)

# Check if the directory exists
if not os.path.exists(directory_path):
    # Create the directory if it doesn't exist
    os.makedirs(directory_path)
    print(f'Directory "{directory_path}" has been created.')
else:
    print(f'Directory "{directory_path}" already exists.')

Directory "data\week1" has been created.


In [13]:
# Specify the file path where you want to save the CSV file
csv_file_path = os.path.join('data', current_week, 'passing.csv')

# Export the DataFrame to a CSV file
passing.to_csv(csv_file_path, index=False)  # Set index=False to exclude the index column

print(f'DataFrame has been exported to {csv_file_path}')

DataFrame has been exported to data\week1\passing.csv
